<a href="https://www.kaggle.com/code/karishmabattina/caloriepred-rforest?scriptVersionId=240650321" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

In [3]:
train_copy, test_copy = train_data, test_data

In [4]:
train_data.sample(5)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
81739,81739,female,31,163.0,61.0,25.0,100.0,40.7,139.0
596019,596019,male,59,175.0,78.0,14.0,91.0,40.2,70.0
730677,730677,female,55,178.0,79.0,18.0,102.0,40.1,104.0
497529,497529,female,36,175.0,75.0,14.0,97.0,40.0,73.0
347315,347315,male,36,195.0,95.0,15.0,103.0,40.3,88.0


In [5]:
test_data.sample(5)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
239447,989447,female,41,147.0,50.0,17.0,104.0,40.2
32768,782768,female,24,143.0,50.0,1.0,81.0,37.5
24209,774209,male,65,175.0,84.0,23.0,97.0,40.8
146352,896352,female,29,171.0,71.0,13.0,89.0,40.3
235172,985172,male,31,162.0,64.0,11.0,96.0,39.9


In [6]:
train_data.shape

(750000, 9)

In [7]:
test_data.shape

(250000, 8)

In [8]:
train_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [9]:
test_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64

In [10]:
def feature_engineering(df):
    df['bmi'] = df['Weight'] / ((df['Height'] / 100) ** 2)    
    df['exercise_intensity'] = df['Heart_Rate'] / df['Duration']
    df['heart_rate_duration'] = df['Heart_Rate'] * df['Duration']
    df['temp_duration'] = df['Body_Temp'] * df['Duration']
    df['hr_to_temp'] = df['Heart_Rate'] / df['Body_Temp']
    df['hr_to_age'] = df['Heart_Rate'] / df['Age']
    df['age_bmi'] = df['Age'] * df['bmi']
    df['max_heart_rate'] = 220 - df['Age']
    df['heart_rate_intensity'] = df['Heart_Rate'] / df['max_heart_rate']
    return df

In [11]:
train_data = feature_engineering(train_data)
test_data = feature_engineering(test_data)

In [12]:
gender_map = {"male" : 0,"female" : 1}
train_data['Sex'] = train_data['Sex'].replace(gender_map)

train_data=train_data.drop(["id"],axis=1)

/tmp/ipykernel_13/4161382762.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace(gender_map)


In [13]:
X = train_data.drop(['Calories'], axis=1)
y = train_data['Calories']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
rf_params = {'n_jobs': -1,
             'n_estimators': 500,
             'min_samples_split': 2,
             'min_samples_leaf': 2,
             'max_features': 'log2',
             'max_depth': 20}

In [17]:
rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)
y_pred = rf_model.predict(X_test)
print(f"RandomForest → R²: {r2_score(y_test, y_pred):.4f}, RMSE: {mean_squared_error(y_test, y_pred, squared=False):.2f}")

RandomForest → R²: 0.9966, RMSE: 3.65


In [18]:
test_data['Sex'] = test_data['Sex'].replace(gender_map)
test_ids = test_data['id']
X_test_final = test_data.drop(["id"], axis=1)


test_predictions = rf_model.predict(X_test_final)

submission = pd.DataFrame({
    'id': test_ids,
    'Calories': test_predictions
})
submission.to_csv('submission.csv', index=False)

/tmp/ipykernel_13/1108437728.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex'] = test_data['Sex'].replace(gender_map)


In [19]:
submission

,id,Calories
0,750000,27.319380
1,750001,106.824884
2,750002,87.631842
3,750003,127.963195
4,750004,75.704519
...,...,...
249995,999995,26.202256
249996,999996,9.273843
249997,999997,73.101117
249998,999998,168.983669
